In [ ]:
%connect

no backend - use %connect


In [ ]:
# Imports
import numpy as np
from mpi4py import MPI
from conduit import Node

no backend - use %connect


In [ ]:
# We already know that we have numerical instability thanks to ascent triggers,
# but we can double check for ourselves. Lets see if we have any negative
# density values:

# Get the MPI communicator
comm = MPI.Comm.f2py(ascent_mpi_comm_id())

# Get each rank's published data
mesh_data = ascent_data().child(0)

# Specifically, retrieve the density values
density_vals = mesh_data["fields/density/values"]

# Compute the minimum density value for each rank
density_min = density_vals.min()

# Allreduce to find the absolute minimum density value
density_min_all = np.zeros(1)
comm.Allreduce(density_min, density_min_all, op=MPI.MIN)

# Print the minimum density value on rank 0
if comm.Get_rank() == 0:
    print(f"Minimum density value: {density_min_all}")

no backend - use %connect


In [ ]:
# In this toy example, we purposefully set the number of timesteps to be too small.
# Lets manually fix the problem without restarting the simulation:

In [ ]:
# First, we query the simulation's current runtime info

output = Node()
jupyter_ascent.execute_callback("getSimInfo", Node(), output)
print(output)

In [ ]:
# Yep, 320 timesteps is too low

In [ ]:
# Second, we manually restore the last stable checkpoint

jupyter_ascent.execute_callback("restorePreviousState", Node(), Node())

In [ ]:
# Third, increase timesteps from 320 to 3200

params = conduit.Node()
params["timesteps"] = 3200
jupyter_ascent.execute_callback("setNumTimesteps", params, Node())

In [ ]:
# Finally, query the simulation's runtime info again to verify our changes

output = Node()
jupyter_ascent.execute_callback("getSimInfo", Node(), output)
print(output)

In [ ]:
%disconnect